# Assignment - 07 

Kavya Shah

SUID:330126500
    
PIN:68 
    
Email: kshah28@syr.edu

# Part-1:

Random forest approach for classification.
Use the same dataset that you used for HW5 
for the classification(It probably won't perform well; say why.)

In [1]:
# Import Modules
import numpy as np
import pandas as pd

In [2]:
# import dataset
dataset = pd.read_csv('61-70.csv')

In [3]:
df = dataset.copy()
df['cit_2017'] = df['cit_2017'].str.replace(",","").astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   univ_rank      100 non-null    int64 
 1   first_initial  100 non-null    object
 2   last_initial   100 non-null    object
 3   cit_2017       100 non-null    int64 
 4   cit_2018       100 non-null    int64 
 5   cit_2019       100 non-null    int64 
 6   cit_2020       100 non-null    int64 
 7   cit_2021       100 non-null    int64 
 8   cit_2022       100 non-null    int64 
 9   h_index        100 non-null    int64 
 10  i_10_index     100 non-null    int64 
dtypes: int64(9), object(2)
memory usage: 8.7+ KB


In [4]:
# Splitting into features and target
# Based on the ratio (citations in 2022)/(citations in 2021),
# approximated to two decimal places, determine the category of each individual as one of the three shown below:
# 1. Low (<1.05).
# 2. Medium (1.06-1.15).
# 3. High (>1.15).

cit_ratio = df['cit_2022']/df['cit_2021'].round(2)
df['cit_ratio'] = cit_ratio.round(2)
# For cit_category = 'Low' : Representation = '0'
df.loc[df['cit_ratio'] <= 1.05, 'cit_category'] = "0" 
# For cit_category = 'Medium' : Representation = '1'
df.loc[(df['cit_ratio'] >= 1.06) & (df['cit_ratio'] <= 1.15), 'cit_category'] = "1"
# For cit_category = 'High' : Representation = '2'
df.loc[df['cit_ratio'] > 1.15, 'cit_category' ] = "2"
df.head()

,univ_rank,first_initial,last_initial,cit_2017,cit_2018,cit_2019,cit_2020,cit_2021,cit_2022,h_index,i_10_index,cit_ratio,cit_category
0,61,S,B,239,291,298,360,447,397,15,17,0.89,0
1,61,C,F,811,774,677,514,562,431,55,107,0.77,0
2,61,J,F,262,279,264,215,208,217,10,10,1.04,0
3,61,D,F,53,54,148,151,280,344,15,16,1.23,2
4,61,L,G,8,35,200,437,560,741,8,7,1.32,2


In [5]:
# Splitting dataset
from sklearn.model_selection import train_test_split

x = df[['cit_2017', 'cit_2018', 'cit_2019', 'cit_2020', 'cit_2021','cit_2022']]
y = df[['cit_category']]

x_train, x_test,y_train, y_test = train_test_split(x,y,train_size = 0.8, test_size=0.2, random_state = 42)

In [6]:
# Normalizing both the train and test data

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [7]:
#  Apply random forest approach for classification

from sklearn.ensemble import RandomForestClassifier

# Creating RF classifier

rand_clf = RandomForestClassifier(n_estimators = 2, random_state=42)
rand_clf.fit(x_train,y_train)
y_pred = rand_clf.predict(x_test)

/Users/kavyashah/anaconda3/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [8]:
y_test

,cit_category
83,0
53,0
70,0
45,0
44,0
39,0
22,0
80,0
10,2
0,0


In [9]:
y_pred

array(['0', '0', '1', '0', '1', '2', '0', '0', '2', '0', '1', '2', '1',
       '0', '0', '2', '1', '0', '0', '0'], dtype=object)

In [10]:
# finding accuracy:
from sklearn import metrics
mse = metrics.mean_squared_error(y_test,y_pred)
print(f'Mean Squared Error: {mse}')
accuracy_score = metrics.accuracy_score(y_test,y_pred)
print(f'Accuracy: {accuracy_score}')

Mean Squared Error: 0.55
Accuracy: 0.75


# Part-2:

Introduce 5 new features
based on the citation numbers. 
and use them in the RF
instead of the citation numbers directly.
Each new feature is:
((citation number in year n+1)-
(citation number in year n))/(citation number in year n) 
for 2016<n<2022.

In [11]:
# add 5 new features:
features = ['cit_2017', 'cit_2018', 'cit_2019','cit_2020', 'cit_2021','cit_2022']

for i in range (0,5):
    new_column_name = f'new_{features[i]}'
    df[new_column_name] = (df[features[i+1]] - df[features[i]]) / df[features[i]]

df.head()

,univ_rank,first_initial,last_initial,cit_2017,cit_2018,cit_2019,cit_2020,cit_2021,cit_2022,h_index,i_10_index,cit_ratio,cit_category,new_cit_2017,new_cit_2018,new_cit_2019,new_cit_2020,new_cit_2021
0,61,S,B,239,291,298,360,447,397,15,17,0.89,0,0.217573,0.024055,0.208054,0.241667,-0.111857
1,61,C,F,811,774,677,514,562,431,55,107,0.77,0,-0.045623,-0.125323,-0.240768,0.093385,-0.233096
2,61,J,F,262,279,264,215,208,217,10,10,1.04,0,0.064885,-0.053763,-0.185606,-0.032558,0.043269
3,61,D,F,53,54,148,151,280,344,15,16,1.23,2,0.018868,1.740741,0.020270,0.854305,0.228571
4,61,L,G,8,35,200,437,560,741,8,7,1.32,2,3.375000,4.714286,1.185000,0.281465,0.323214


In [12]:
x_new = df[['new_cit_2017','new_cit_2018','new_cit_2019','new_cit_2020','new_cit_2021']]

In [13]:
# Splitting dataset for Part-2
x_new_train, x_new_test, y_new_train, y_new_test = train_test_split(x_new, y, train_size=0.8, test_size=0.2, random_state=42)

In [14]:
# Handling infinite and missing values
x_new_train.replace([np.inf, -np.inf], 0, inplace=True)
# x_new_train.replace([np.nan, -np.nan], 0, inplace=True)
x_new_test.replace([np.inf,-np.inf], 0 , inplace = True)
# x_new_train.dropna(inplace=True)

In [15]:
# Normalization

scaler = MinMaxScaler(feature_range=(0, 1))
x_new_train = scaler.fit_transform(x_new_train)
x_new_test = scaler.transform(x_new_test)

In [16]:
# Applying Random Forest
rand_clf.fit(x_new_train,y_new_train)
y_new_pred = rand_clf.predict(x_new_test)

/Users/kavyashah/anaconda3/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [17]:
y_new_test

,cit_category
83,0
53,0
70,0
45,0
44,0
39,0
22,0
80,0
10,2
0,0


In [18]:
y_new_pred

array(['0', '0', '0', '0', '0', '0', '0', '0', '2', '0', '1', '2', '1',
       '0', '2', '2', '1', '0', '0', '0'], dtype=object)

In [19]:
# finding accuracy:
mse_new = metrics.mean_squared_error(y_new_test,y_new_pred)
print(f'Mean Squared Error: {mse_new}')
accuracy_score_new = metrics.accuracy_score(y_new_test,y_new_pred)
print(f'Accuracy: {accuracy_score_new}')

Mean Squared Error: 0.05
Accuracy: 0.95


In [20]:
# n estimators | mse1 | mse2 

# random State = 42
# 1 | 1.55 | 0.4 
# 2| 0.55 | 0.05 
# 3|1.15 | 0.1 
# 50|1.2 | 0.0 
# 75|1.0 |0.0 
# 100| 1.0 | 0.0 
# 500|1.0 | 0.0
# 1000|1.15 | 0.0 

# random state = 1
# 1 | 1.45 | 0.0 
# 2| 0.85 | 0.0
# 3|1.15 | 0.1 
# 50|0.95 | 0.1 
# 75|1.0 |0.1 
# 100| 1.2 | 0.1 
# 500|1.0 | 0.0
# 1000|1.15 | 0.0 


# Conclusion:

Part 1:

Random forest approach for classification. Use the same dataset that you used for HW5 for the classification(It probably won't perform well; say why.)

Answer:

- Original Citation numbers provide total counts, but they might not capture the nuanced changes or trends over time, Hence the model might not know if the research paper is gaining popularity or lossing it as it only considers the absolute numbers. 

- Another reason might be that the counts from different years could be on different scales, and this may affect the model's ability to generalize patterns. However, the percentage change that we do in part 2 and take them as features potentially improves the model's ability to recognize patterns.

- Outliers can also be a reason, such as sudden spike in citations in single year

- The model might benifit from features that explicitly highlight changes over time, as provided in part 2 rather than just values.

- The reason that HW 05 (85% Accuracy) is better than Part 1 in accuracy can be because the data might be better suited for the learning capabilities of neural network, which can adapt to complexities and non-linearities in the relationship between citation numbers and categories.




Part 2:

In part 2, the imporved performance of our new approach i.e. introducing 5 new features, which considers the percentage change in citation numbers over consequtive years can be attributed to its ability to capture dynamic patterns and trends in the popularity of research papers. Unlike the traditional method that focuses on the total number of citations, Part 2 looks at how this count changes over time, enabling the model to identify subtle variations and nuanced relationships between citation patterns and categories. By incorporating this perspective, our model gains more understanding of the data, making it more adept at predicting the correct categories of research papers. Hence, the accuracy of part 2 is between 95% to 100% no matter what estimator or random_state we take.